#### state_postal_file = "CSV_data/state_postal_code.csv"
state,postal_code
Alabama ,AL

#### shootings_file = "CSV_data/fatal_police_shootings_data(redundant).csv"
Victim's age,Victim's gender,Victim's race,Date of Incident (month/day/year),City,State,County,Cause of death,A brief description of the circumstances surrounding the death,Official disposition of death (justified or other),Criminal Charges?,Symptoms of mental illness?,Unarmed/Did Not Have an Actual Weapon,Alleged Weapon (Source: WaPo and Review of Cases Not Included in WaPo Database),Alleged Threat Level (Source: WaPo),Fleeing (Source: WaPo)
44,Male,Unknown race,10/28/2020,Trigg County,KY,Trigg,Gunshot,"Officers were called to the report of a man cutting a hole in the floor of his home with a chainsaw. When an officer arrived, they got into an altercation with Biby. As a result of the altercation, the officer discharged his service weapon, striking Biby. Biby was later pronounced dead at the Trigg County hospital. Sheriff Aaron Acree was allegedly involved in the shooting. ",Pending investigation,No known charges,No,Allegedly Armed,undetermined,undetermined,

#### pres_file = "CSV_data/president_county_candidate.csv"
state,county,candidate,party,voteStates
Delaware,Kent County,Joe Biden,DEM,44518

In [450]:
import sklearn as sk
import pandas as pd
import numpy as np
import csv
import time
import math
import datetime
from collections import defaultdict

In [198]:
# load folders
shootings_file = "CSV_data/fatal_police_shootings_detailed.csv"
pres_file = "CSV_data/president_county_candidate.csv"
state_postal_file = "CSV_data/state_postal_code.csv"

shootings = pd.read_csv(shootings_file)
presidency = pd.read_csv(pres_file)
state_postal = pd.read_csv(state_postal_file)

In [199]:
# turn all strings in dataframe to lowercase
presidency = presidency.applymap(lambda s:s.lower() if type(s) == str else s)
state_postal = state_postal.applymap(lambda s:s.lower() if type(s) == str else s)
shootings = shootings.applymap(lambda s:s.lower() if type(s) == str else s)

In [200]:
state_postal_dic = {row["postal_code"]:row['state']  for i, row in state_postal.iterrows()}

# replace every state postal to state
# al -> alabama
shootings['state'] = shootings['state'].apply(lambda s: state_postal_dic.get(s, ""))

In [201]:
# turn all dates to timestamps
s = shootings['date'][0]
def process_date(date):
    if len(date) == 0:
        return 0

    dt = datetime.datetime.strptime(date, "%m/%d/%Y").timetuple()
    return time.mktime(dt)

In [202]:
# county_state, votes_rep, votes_dem, timestamp, cause_of_death_index, criminal_charges_index, mental_illness_index, weapon_type, threat_level, fleeing_index]
[1, 14124, 124, 1420153200, 1, 1, 1, 1, 1, 1, 1]

# TODO turn postal to state in shootings
# TODO: map (state, county) to specific index
# TODO: turn date to timestamp
# TODO: create a cause of death index
# TODO: create criminal charges index from set
# TODO: create mental ilness - mental ilness aready has a YES/NO specified
# TODO: create weapon type index
# TODO: create threat level index
# TODO: create fleeing index

[1, 14124, 124, 1420153200, 1, 1, 1, 1, 1, 1, 1]

In [203]:
# Remove duplicates from state and county
presidency_no_dup = presidency.drop_duplicates(subset=['state', 'county'])

def process_state_county(value):
    state = row['state']
    county = row['county'].replace("county", "").strip()

    return (state, county)

# Turn every (state, county) tuple into a unique index
state_county_idx = defaultdict(int)
for i, row in presidency_no_dup.iterrows():
    key = process_state_county(row)
    # index starts with 1 :O
    state_county_idx[key] = i + 1

In [204]:
cause_of_death_idx = {cause:i+1 for i,cause in enumerate(set(shootings['death']))}
charges_idx = {charges:i+1 for i,charges in enumerate(set(shootings['charges']))}
mental_illness_idx = {"yes":0, "no":1, "unknown": 2}
weapon_idx = {weapon:i+1 for i,weapon in enumerate(set(shootings['weapon']))}
weapon_type_idx = {weapon:i+1 for i,weapon in enumerate(set(shootings['weapon_type']))}
threat_idx = {threat:i+1 for i,threat in enumerate(set(shootings['threat']))}

In [462]:
# Fleeing index is a bit different
fleeing_set = set(shootings['fleeing'])
print(fleeing_set)

# We're going to set the data to 0 or 1 based on if he was fleeing or wasn't
def fleeing_idx_func(value):
    if value in ['car', 'foot']:
        return 1
    else:
        return 0

county_id_file = "CSV_data/county_code.csv"
county = pd.read_csv(county_id_file)
county = county.applymap(lambda s:s.lower() if type(s) == str else s)

only_state = county.loc[county['code'] % 1000 == 0]

keys = list(only_state['code'].values)
only_county = county[~county['code'].isin(keys)]


def get_county_code(state_name, county_name):
    state_name = str(state_name).strip()
    county_name = str(county_name).strip()
    if county_name == "nan":
        return 0


    state_code = only_state[only_state['name'].str.contains(state_name)]
    if len(state_code) == 0:
        return 0

    state_code = state_code["code"].values[0]

    name_bol = only_county['name'].str.contains(county_name)
    range_bol = only_county["code"].between(state_code, state_code + 999)
    county_code = only_county[name_bol & range_bol]

    return 0 if len(county_code) == 0 else county_code["code"].values[0]


{nan, 'not fleeing', 'car', 'foot', 'other'}


In [474]:
left_count = defaultdict(int)
right_count = defaultdict(int)

left = ["dem"]
right = ["rep"]


for index, row in presidency.iterrows():
    state_county = (row["state"], row["county"])

    key = get_county_code(row["state"], row["county"])
    party = row["party"]

    votes = row["voteStates"]

    if party in left:
        left_count[key] += votes
    elif party in right:
        right_count[key] += votes

In [475]:

# TODO if county not specified, find out based on state and city
after_2016_elections = process_date("11/07/2016")
processed_data = []
for i, row in shootings.iterrows():
     state_county = (row["state"], row["county"])

     state_county_ids = get_county_code(row["state"], row["county"])
     date = process_date(row["date"])

     # this will skip anything before 2016 elections, because we
     # dont have the data
     if date < after_2016_elections:
         continue

     death = cause_of_death_idx[row["death"]]
     charges = charges_idx[row["charges"]]
     illness = mental_illness_idx.get(row["mental_illness"], mental_illness_idx["unknown"])
     weapon = weapon_idx[row["weapon"]]
     weapon_type = weapon_type_idx[row["weapon_type"]]
     threat = threat_idx[row["threat"]]
     fleeing = fleeing_idx_func(row["fleeing"])

     left_count_n = left_count.get(state_county_ids,0 )
     right_count_n = right_count.get(state_county_ids, 0)

     values = (state_county_ids, date, death, charges, illness, weapon, weapon_type, threat, fleeing, left_count_n, right_count_n)
     print(values)
     processed_data.append(values)

(21221, 1603839600.0, 16, 21, 1, 3, 12, 2, 0, 1791, 5487)
(0, 1603753200.0, 16, 21, 1, 3, 72, 3, 0, 6631488, 4374328)
(0, 1603753200.0, 16, 21, 1, 3, 12, 2, 0, 6631488, 4374328)
(0, 1603666800.0, 16, 21, 1, 3, 72, 4, 1, 6631488, 4374328)
(0, 1603666800.0, 16, 21, 0, 3, 106, 3, 0, 6631488, 4374328)
(0, 1603576800.0, 16, 21, 1, 3, 72, 4, 0, 6631488, 4374328)
(0, 1603490400.0, 16, 21, 1, 1, 12, 2, 0, 6631488, 4374328)
(18029, 1603490400.0, 16, 21, 1, 3, 106, 4, 0, 5433, 19441)
(0, 1603404000.0, 16, 21, 1, 3, 72, 4, 0, 6631488, 4374328)
(0, 1603404000.0, 16, 21, 1, 3, 72, 4, 1, 6631488, 4374328)
(0, 1603404000.0, 16, 21, 1, 3, 72, 4, 1, 6631488, 4374328)
(5097, 1603404000.0, 16, 21, 1, 3, 72, 3, 1, 3869, 12789)
(54087, 1603317600.0, 16, 21, 1, 3, 12, 3, 0, 1451, 4199)
(0, 1603317600.0, 16, 21, 0, 3, 72, 4, 0, 6631488, 4374328)
(0, 1603317600.0, 16, 21, 1, 3, 72, 4, 0, 6631488, 4374328)
(4021, 1603317600.0, 16, 21, 1, 3, 72, 4, 1, 65355, 88597)
(12081, 1603231200.0, 16, 21, 1, 3, 72, 4, 0, 

In [476]:
data = pd.DataFrame.from_records(processed_data)
data.to_csv("./test.csv")

In [477]:
id = get_county_code("ohio", "hamilton")
left_count.get(id, 0)

238101